## PCX - Phuture Crypto Market Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = None

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    ids_to_replace=[['bitcoin'],['wrapped-bitcoin']],
    remove_category_assets=['staked-ether','wrapped-bitcoin','rocket-pool-eth','frax-ether','staked-frax-ether']
    )
liquidity

Excluding mantle, pricing data available only for 144 < 180 days
Excluding beam-2, pricing data available only for 43 < 180 days
0x0000000000000000000000000000000000001010
0xbec243c995409e6520d7c41e404da5deba4b209b
0x925206b8a707096ed26ae47c84747fe0bb734f59
0x19de6b897ed14a376dda0fe53a5420d2ac828a28

0x7205705771547cf79201111b4bd8aaf29467b9ec
Dropping whitebit because a price feed is unavailable


,spot price,delivery price,slippage,blockchain
id,,,,
illuvium,1.005268e-02,0.010082,0.002927,ethereum
ethereum,4.445825e-04,0.000445,0.000871,ethereum
uniswap,1.639937e-01,0.164054,0.000370,arbitrum-one
wrapped-bitcoin,2.282000e-05,0.000023,0.000000,arbitrum-one
binancecoin,4.331810e-03,0.004329,-0.000536,binance-smart-chain
maker,6.947711e-04,0.000694,-0.000703,ethereum
matic-network,1.200742e+00,1.199867,-0.000729,polygon-pos
chainlink,6.459242e-02,0.064523,-0.001074,ethereum
avalanche-2,3.832142e-02,0.038266,-0.001436,avalanche


## Show Results

In [4]:
results

,name,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
wrapped-bitcoin,Bitcoin,857650691861,0.311197,80,0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f,arbitrum-one
ethereum,Ethereum,269765837090,0.311197,80,ETH,ethereum
binancecoin,BNB,35509856004,0.153857,40,BNB,binance-smart-chain
avalanche-2,Avalanche,9542698411,0.041347,11,AVAX,avalanche
chainlink,Chainlink,8591678322,0.037226,10,0x514910771af9ca656af840dff83e8264ecf986ca,ethereum
matic-network,Polygon,7723658203,0.033465,9,0x0000000000000000000000000000000000001010,polygon-pos
shiba-inu,Shiba Inu,5779644452,0.025042,7,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,ethereum
uniswap,Uniswap,4592920791,0.019900,6,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0,arbitrum-one
lido-dao,Lido DAO,2034991314,0.008817,2,0x5a98fcbea516cf06857215779fd812ca3bef1b32,ethereum
